In [21]:
%load_ext autoreload
%autoreload 2

import os
from Bio import PDB
import numpy as np
import pandas as pd
from pathlib import Path
import py3Dmol
import warnings
import Bio
from Bio.PDB import PDBParser, Superimposer

from benchmark import visualize_structure_alignment, plot_with_error

warnings.simplefilter("ignore", Bio.PDB.PDBExceptions.PDBConstructionWarning)


pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
def compute_rmsd(pdb_path_1: str, pdb_path_2: str, model_index: int = 0) -> float:
    """
    Compute the RMSD between two PDB files based on their CA atoms.

    Args:
        pdb_path_1 (str): Path to first PDB file.
        pdb_path_2 (str): Path to second PDB file.
        model_index (int): Index of the model to use if multiple models are present.

    Returns:
        float: RMSD value.
    """
    parser = PDBParser(QUIET=False)

    structure1 = parser.get_structure("structure1", pdb_path_1)
    structure2 = parser.get_structure("structure2", pdb_path_2)

    atoms1 = []
    atoms2 = []

    for res1, res2 in zip(
        structure1[model_index].get_residues(), structure2[model_index].get_residues()
    ):
        if res1.has_id("CA") and res2.has_id("CA"):
            atoms1.append(res1["CA"])
            atoms2.append(res2["CA"])

    if len(atoms1) != len(atoms2):
        raise ValueError(f"Residue mismatch: {len(atoms1)} != {len(atoms2)}")

    sup = Superimposer()
    sup.set_atoms(atoms1, atoms2)
    return sup.rms


compute_rmsd(
    "./data/bio2token_casp/T1104-D1_bio2token.pdb",
    "./data/casp15_targets_TSdomains_4invitees/T1104-D1.pdb",
)


np.float64(0.7375210365317113)

In [55]:
truth_dir = "data/casp15_targets_TSdomains_4invitees"
pred_dirs = [
    # "data/casp15_targets_TSdomains_4invitees",
    "data/bio2token_casp",
    "data/foldtoken_level5_casp",
    "data/foldtoken_level6_casp",
    "data/foldtoken_level7_casp",
    "data/foldtoken_level8_casp",
    "data/foldtoken_level9_casp",
    "data/foldtoken_level10_casp",
    "data/foldtoken_level12_casp",
]

parser = PDB.PDBParser()
truth_files = [f.split(".")[0] for f in os.listdir(truth_dir) if f.endswith(".pdb")]
truth_files.sort()

all_results = {}
for pred_dir in pred_dirs:
    pred_files = [f.split(".")[0] for f in os.listdir(pred_dir) if f.endswith(".pdb")]
    pred_files.sort()

    model_name = os.path.basename(pred_dir).replace("_casp", "")
    for truth_file, pred_file in zip(truth_files, pred_files):
        if truth_file.split(".")[0] != pred_file.split("_")[0]:
            raise ValueError(
                f"Truth file {truth_file} does not match prediction file {pred_file}"
            )
        else:
            target = truth_file.split(".")[0]
        try:
            rmsd = compute_rmsd(
                os.path.join(pred_dir, pred_file + ".pdb"),
                os.path.join(truth_dir, truth_file + ".pdb"),
            )
        except Exception as e:
            print(f"Error in RMSD calculation: {e}")
            rmsd = None

        if target not in all_results:
            all_results[target] = {}
        all_results[target][f"{model_name}_RMSD"] = rmsd

df = pd.DataFrame.from_dict(all_results, orient="index")
display(df)

Error in RMSD calculation: Residue mismatch: structure1 has 147 residues, structure2 has 148 residues
Error in RMSD calculation: Residue mismatch: structure1 has 113 residues, structure2 has 115 residues
Error in RMSD calculation: Residue mismatch: structure1 has 139 residues, structure2 has 141 residues
Error in RMSD calculation: Residue mismatch: structure1 has 149 residues, structure2 has 150 residues
Error in RMSD calculation: Residue mismatch: structure1 has 47 residues, structure2 has 48 residues
Error in RMSD calculation: Residue mismatch: structure1 has 47 residues, structure2 has 48 residues
Error in RMSD calculation: Residue mismatch: structure1 has 47 residues, structure2 has 48 residues
Error in RMSD calculation: Residue mismatch: structure1 has 47 residues, structure2 has 48 residues
Error in RMSD calculation: Residue mismatch: structure1 has 47 residues, structure2 has 48 residues
Error in RMSD calculation: Residue mismatch: structure1 has 47 residues, structure2 has 48 r

,bio2token_RMSD,foldtoken_level5_RMSD,foldtoken_level6_RMSD,foldtoken_level7_RMSD,foldtoken_level8_RMSD,foldtoken_level9_RMSD,foldtoken_level10_RMSD,foldtoken_level12_RMSD
T1104-D1,0.737521,1.603654,1.380800,1.298154,0.966320,0.857345,0.820845,0.711739
T1106s1-D1,0.894732,2.235067,1.576464,1.694806,1.320534,1.098733,0.930388,1.077418
T1106s2-D1,0.724124,1.421836,1.411146,0.868377,0.822946,0.986992,0.912395,0.716482
T1109-D1,0.706411,2.772142,1.440362,1.801300,1.262320,1.122182,1.129094,1.081476
T1110-D1,0.684899,2.653970,2.027106,1.340347,1.325064,1.114246,1.369690,1.070269
T1112-D1,0.903431,4.641957,2.974979,2.605774,2.724899,2.286424,2.518600,2.502462
T1113-D1,0.883217,3.718407,2.969302,3.133844,3.135323,2.114748,2.257757,2.559514
T1114s1-D1,0.952594,0.968053,1.032799,1.075391,1.693519,1.922422,1.195107,1.469920
T1114s2-D1,0.815802,3.899639,3.332826,2.496847,2.011913,1.804377,1.621344,1.636948
T1114s3-D1,0.858504,3.279127,2.488172,1.913892,1.589001,1.534294,1.278400,1.339736


In [67]:
df.columns = [
    "bio2token",
    "foldtoken 32",
    "foldtoken 64",
    "foldtoken 128",
    "foldtoken 256",
    "foldtoken 512",
    "foldtoken 1024",
    "foldtoken 4096",
]


df.to_csv("./data/bio2token_casp_rmsd_results.csv", index=True)

---


In [40]:
casp_metadata = pd.read_csv("./data/casp_metadata.csv", delimiter="\t").drop(
    columns=["#"]
)
rmsd_results = pd.read_csv("./data/bio2token_casp_rmsd_results.csv", index_col=0)

casp_metadata = casp_metadata[casp_metadata["Classification"].str.fullmatch("FM")]

display(casp_metadata)

filtered_indices = [
    idx
    for idx in rmsd_results.index
    if any(target in idx for target in casp_metadata["Target"])
]
rmsd_results = rmsd_results.loc[filtered_indices]

display(rmsd_results)
df = rmsd_results

,Target,Residues,Type,Domains,Residues in domain,Classification,PDB
0,T1104,117,All groups,T1104-D1: 1-117,117,FM,7roa
6,T1113,193,All groups,T1113-D1: 1-192,192,FM,7uyx
14,T1120,235,All groups,T1120-D1: 8-125,118,FM,7qvb
15,T1120,235,All groups,T1120-D2: 126-235,110,FM,7qvb
16,T1121,381,All groups,T1121-D1: 2-204,203,FM,7til
18,T1122,241,All groups,T1122-D1: 4-237,234,FM,8bbt
21,T1125,1200,All groups,T1125-D1: 327-460,134,FM,8h2n
22,T1125,1200,All groups,T1125-D2: 461-608,148,FM,8h2n
24,T1125,1200,All groups,T1125-D4: 798-946,149,FM,8h2n
25,T1125,1200,All groups,T1125-D5: 947-1096,150,FM,8h2n


,bio2token,foldtoken 32,foldtoken 64,foldtoken 128,foldtoken 256,foldtoken 512,foldtoken 1024,foldtoken 4096
T1104-D1,0.737521,1.603654,1.380800,1.298154,0.966320,0.857345,0.820845,0.711739
T1113-D1,0.883217,3.718407,2.969302,3.133844,3.135323,2.114748,2.257757,2.559514
T1120-D1,0.755351,1.607281,1.600221,1.558989,1.238299,1.035735,1.335048,1.105757
T1120-D2,0.795090,1.926442,1.432745,0.949585,1.028857,0.922227,0.887018,0.903988
T1121-D1,0.890124,3.795151,2.488831,1.484729,2.025722,1.663299,1.418762,1.399452
T1121-D2,0.945058,2.066247,1.656643,1.425744,1.577678,1.387708,1.446935,1.193631
T1122-D1,0.944706,5.439527,2.644547,2.002693,2.395990,1.994762,1.641277,1.989896
T1125-D1,0.802187,3.084521,2.200077,1.110489,1.252971,1.202333,1.055077,0.914587
T1125-D2,NaN,2.011476,1.501738,1.451648,1.206075,0.901925,0.984406,0.902483
T1125-D3,NaN,1.991400,1.799675,1.108439,1.102238,0.930443,0.928630,1.183834


In [41]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.boxplot(data=df.dropna(how="any"), showfliers=False)
plt.xticks(rotation=45, ha="right")
plt.ylabel("RMSD")
plt.title(
    "Distribution of CASP 15 (FM only) Backbone Reconstruction RMSD Values by Model (side chains omitted if present)"
)
plt.tight_layout()
plt.show()


In [39]:
df.describe()  # .to_csv("./data/bio2token_casp_rmsd_results_description.csv", index=True)

,bio2token,foldtoken 32,foldtoken 64,foldtoken 128,foldtoken 256,foldtoken 512,foldtoken 1024,foldtoken 4096
count,92.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000
mean,0.982109,4.812170,3.494836,2.395314,2.371472,2.212014,2.088312,2.078897
std,0.551778,6.218504,5.921372,2.693853,3.201545,3.048829,2.934474,2.808976
min,0.635363,0.968053,0.935893,0.801132,0.694670,0.721484,0.775155,0.681697
25%,0.753712,2.155563,1.597269,1.368284,1.245635,1.050804,1.005887,1.024788
50%,0.817384,2.911908,1.981677,1.602530,1.534131,1.346927,1.281883,1.200789
75%,0.927806,4.255589,2.585290,2.325924,1.955802,1.917388,1.631311,1.723598
max,3.673086,45.461208,45.092077,20.115543,20.976147,21.219458,19.679417,20.163179
